### Q1. Running Elastic

What's the version.build_hash value?

Result of `curl http://localhost:9200`:

In [66]:
result = {
  "name" : "0a6727f25504",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "GUzwBIDFQsWjJ2_V-MJDwQ",
  "version" : {
    "number" : "9.0.1",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "73f7594ea00db50aa7e941e151a5b3985f01e364",
    "build_date" : "2025-04-30T10:07:41.393025990Z",
    "build_snapshot" : False,
    "lucene_version" : "10.1.0",
    "minimum_wire_compatibility_version" : "8.18.0",
    "minimum_index_compatibility_version" : "8.0.0"
  },
  "tagline" : "You Know, for Search"
}

In [67]:
result['version']['build_hash']

'73f7594ea00db50aa7e941e151a5b3985f01e364'

#### Ans: '73f7594ea00db50aa7e941e151a5b3985f01e364'

In [2]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

Which function do you use for adding your data to elastic?

- insert
- index
- put
- add

#### Ans: index

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('htttp://localhost:9200')

In [5]:
es_client.info()

ObjectApiResponse({'name': '0a6727f25504', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GUzwBIDFQsWjJ2_V-MJDwQ', 'version': {'number': '9.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '73f7594ea00db50aa7e941e151a5b3985f01e364', 'build_date': '2025-04-30T10:07:41.393025990Z', 'build_snapshot': False, 'lucene_version': '10.1.0', 'minimum_wire_compatibility_version': '8.18.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

### Q3. Searching

Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

- 84.50
- 64.50
- 44.50
- 24.50

In [9]:
query = "How do execute a command on a Kubernetes pod?"

In [16]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [14]:
search_response = es_client.search(index=index_name, body=search_query)
search_response

ObjectApiResponse({'took': 18, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': '1PGtPJcBo5vjBxHtMk1S', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'Y_GtPJcBo5vjBxHtNE5o', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubern

#### Ans: top score is approx. 44.5

### Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?

In [20]:
query = "How do copy a file to a Docker container?"

In [23]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },"filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [24]:
search_response = es_client.search(index=index_name, body=search_query)

In [26]:
hits = search_response['hits']['hits']
result_docs = [hit['_source'] for hit in hits]
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [28]:
print(result_docs[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


#### Ans: Third Q returned is 'How do I copy files from a different folder into docker container’s working directory?'

### Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

In [29]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [44]:
context = ""
for doc in result_docs:
    context += f"Q:{doc['question']}\n\nA:{doc['text']}\n\n"

In [45]:
print(context)

Q:How do I debug a docker container?

A:Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I copy files from my local machine to docker container?

A:You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q:How do I copy files from a different folder into docker container’s working directory?

A:You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do 

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

In [46]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [47]:
prompt = prompt_template.format(question=query, context=context).strip()

In [48]:
print(f'\n prompt:\n{prompt}\n\n len: {len(prompt)}')


 prompt:
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do copy a file to a Docker container?

CONTEXT:
Q:How do I debug a docker container?

A:Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q:How do I copy files from my local machine to docker container?

A:You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in

What's the length of the resulting prompt? (use the len function)

- 946
- 1446
- 1946
- 2446

#### Ans: closest ans is 1446

### Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:
`pip install tiktoken`

Let's calculate the number of tokens in our query:
`encoding = tiktoken.encoding_for_model("gpt-4o")`

In [51]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [54]:
prompt_tokens = encoding.encode(prompt)
len(prompt_tokens)

320

Use the encode function. How many tokens does our prompt have?

- 120
- 220
- 320
- 420

#### Ans: 320 token in the prompt

### Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?

Note: you can replace OpenAI with Ollama. See module 2.

In [58]:
from openai import OpenAI
def llm(prompt, *args, **kwargs):
    client = OpenAI()
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
    )
    return response.choices[0].message.content    

In [59]:
response = llm(prompt)

In [60]:
print(response)

To copy a file from your local machine into a Docker container, use the `docker cp` command. The basic syntax is:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```

This command allows you to specify the file or directory on your local machine and the destination path inside the running Docker container.


### Bonus: calculating the costs (ungraded)

The Realtime API using text and gpt-4o charges 5.00 / 1M input tokens and 20.00 / 1M output tokens, as of 6/4/2025

Part 1:
Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

In [62]:
total_input_tokens = 1000*150
total_output_tokens = 100*250
total_cost = (5*total_input_tokens + 20*total_output_tokens)/1000000
total_cost

1.25

#### Ans: total cost is 1.25

Part 2: redo the calculations (cost of 1000 requests) with the values obtained for our actual prompt in the prior exercises. Note: I will use the len of my prompt rather than the closest answer choice given.

In [64]:
total_input_tokens = 1000*1443
total_output_tokens = 100*320
total_cost = (5*total_input_tokens + 20*total_output_tokens)/1000000
total_cost

7.855

#### Ans: total cost 7.86